In [3]:
import tensorflow as tf
import tensorflow_recommenders as tfrs
import keras
import pandas as pd
import numpy as np
from typing import Dict, Text
EMB_DIM=32


In [2]:
# Loading in the full dataset
dataset=pd.DataFrame()
for i in range(4):
    df = pd.read_csv(f"../MIND_large/csv/tensorflow_dataset_chunk{i}.csv", index_col=0)
    dataset = pd.concat([dataset, df])
dataset


,user_id,time,news_id,category,sub_category,title,abstract,interaction_type,score
0,U66319,1,N10721,entertainment,entertainment-celebrity,Mike Johnson asks out Keke Palmer after Demi L...,Mike Johnson tried to ask out Keke Palmer in a...,history,1
1,U66319,1,N128129,movies,movies-celebrity,Brie Larson Has the Best Reaction Ever After T...,The 'Captain Marvel' star was left speechless ...,history,1
2,U66319,1,N28406,news,newsworld,Accused dine-and-dashers in viral video at St....,Five young black men who posted a video of a m...,history,1
3,U66319,1,N118998,news,newsgoodnews,Trooper pulls over to save flag on highway,The trooper is being praised for stopping his ...,history,1
4,U66319,1,N38884,sports,mma,UFC champ Khabib Nurmagomedov seen training in...,Khabib Nurmagomedov doesn't mess around.,history,1
...,...,...,...,...,...,...,...,...,...
20645888,U491432,0,N87192,finance,finance-companies,Bill Gates tops Jeff Bezos as world's richest ...,This time it's official.,impression,0
20645889,U491432,0,N31918,news,newsworld,Oscar Wilde's stolen ring found by Dutch 'art ...,A golden ring once given as a present by the f...,impression,0
20645890,U491432,0,N73556,sports,football_nfl,Report: At least 24 teams expected to attend K...,It looks like the majority of the teams in the...,impression,0
20645891,U491432,0,N92223,lifestyle,lifestylebuzz,This 10-Year-Old Girl's Demanding Christmas Wi...,A father recently shared his 10-year-old daugh...,impression,0


In [4]:
dataset = dataset[dataset.index < 10000000]
dataset['score'] = dataset['score'].astype('float32')


/tmp/ipykernel_519030/64627017.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['score'] = dataset['score'].astype('float32')


In [5]:
# removing the NaN values from the dataset and changing the dataset type into a tensorflow dataset.
dataset.dropna(inplace=True)
news = pd.read_csv("../MIND_large/csv/news.csv", index_col=0)
news.drop(columns=['url', 'title_entities', 'abstract_entities'], inplace=True)
news.dropna(inplace=True)
tf_ds = tf.data.Dataset.from_tensor_slices(dict(dataset))
catalog_ds = tf.data.Dataset.from_tensor_slices(dict(news))

/tmp/ipykernel_519030/2952817448.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.dropna(inplace=True)


In [6]:
# Mapping all features to relevant items in the map dataset
ratings_ds = tf_ds.map(lambda x : {
    'user_id' : x['user_id'],
    'news_id' : x['news_id'],
    'time' : x['time'],
    'category' : x['category'],
    'sub_category' : x['sub_category'],
    'title' : x['title'],
    'abstract' : x['abstract'],
    'score' : x['score']
})


In [7]:
# Mapping all features to relevant items in the map dataset
news_ds = catalog_ds.map(lambda x : {
    'news_id' : x['news_id'],
    'category' : x['category'],
    'sub_category' : x['sub_category'],
    'title' : x['title'],
    'abstract' : x['abstract']
})

In [8]:
# Creating lists of all unique user and news ids
unique_news_ids = np.unique(np.concatenate(list(news_ds.batch(1000).map(lambda x : x['news_id']))))
unique_categories = np.unique(np.concatenate(list(ratings_ds.batch(1000).map(lambda x : x['category']))))
unique_subcategories = np.unique(np.concatenate(list(ratings_ds.batch(1000).map(lambda x : x['sub_category']))))



unique_user_ids = np.unique(np.concatenate(list(ratings_ds.batch(1000).map(lambda x : x['user_id']))))
timestamp_vals = np.unique(np.concatenate(list(ratings_ds.batch(1000).map(lambda x: x['time']))))

# Output looks like array([b'U1', b'U100', b'U1000', ..., b'U99993', b'U99994', b'U99998'], dtype=object)

2024-03-21 05:27:30.052511: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-03-21 05:30:03.430969: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-03-21 05:32:27.713398: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-03-21 05:35:01.786033: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-03-21 05:37:17.128894: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


# Tensorflow Modelling
To finish our implementations of recommender systems we decided to experiment with Tensorflows recommender system library. Tensorflow features robust set of tools to build complex models that we were hoping to experiment more with. Unfortunately due to a bug with the library for the retrieval task, we were unable to implement a complete recommender system. Regardless, we did create a multistage ranking model for a larger recommender system that could rate items for users.

## Ranking Models
The most simple models at our disposal are single ranking or retreival models.

### Ranking
Given that we have our data in a compatible format, we can initialize a simple ranking model. The ranking model uses the keras model as its super, in doing so we allow ourselves to incorporate our model into the keras ecosystem, define custom behavior and additionally gain access to features present in keras models like specific optimizers. 

The basic ranking model is comprised of seveeral smaller keras sequential models focused on generating embeddings for features.

#### Embedding Models
Most embedding models are comprised of 2 layers: string lookup and embedding. The string lookup layer maps items on to unique indices utilizing the unique ID vocabularies generated during the initial processing of data. This index is then passed on to an embedding layer which creates dense vector representations of it using the previously defined embedding dimension. As the model trains these embeddings are updated to improve performance.

Two separate embedding models are used to handle vectorizing titles and abstract with similar model definitions. Conveniently, these embedding layers generalize well to unseen data through the use of out of vocabulary tokens. Out of vocabulary tokens, or OOV, are generated when a new user or item is placed in the embedding model, conveniently OOV still recieve embeddings. 

#### Ratings Model
The ratings model is another keras sequential model comprised of dense layers. For clarification, a dense layer is a layer of neurons that is fully connected to both preceeding and following dense layers if applicable. The first dense layers contain a large number of neurons and utilize the rectified linear unit activation function to introduce non-linearity into the data. The final layer has an output of dimension 1; this dimension is 1 since we are looking to predict a ranking.

#### The Call Function
The call function uses the sequential models present at initialization on data from the dataset to generate embeddings for the ids present, and then predict the rating given the embeddings. 

In [42]:
class UserModel(keras.Model):
    def __init__(self, embedding_dimension : int = 32):
        super().__init__()

        # User and article embedding models. 
        # Define the user ID model.
        self.user_embeddings = keras.Sequential([
            # Here we exclude the mask token as to better handle OOV items like new users or items.
            keras.layers.StringLookup(vocabulary = unique_user_ids),

            # Our final layer in the user ID model is an embedding layer which takes the IDs index
            # and creates a dense vector representation of it.
            keras.layers.Embedding(input_dim = len(unique_user_ids) + 1, output_dim = embedding_dimension)
        ])

        
        self.timestamp_embedding = keras.Sequential([
            keras.layers.Embedding(len(timestamp_vals) + 2, 32)
        ])

    def call(self, inputs):
        return tf.concat([
            self.user_embeddings(inputs['user_id']),
            self.timestamp_embedding(inputs['time'])
        ], axis=1)

class QueryModel(keras.Model):
    def __init__(self, layer_sizes):

        super().__init__()

        self.embedding_model = UserModel()

        self.dense_layers = keras.Sequential()

        for layer_size in layer_sizes[:-1]:
            self.dense_layers.add(keras.layers.Dense(layer_size, activation="relu"))
        
        self.dense_layers.add(keras.layers.Dense(layer_sizes[-1]))

    def call(self, inputs):
        feature_embedding = self.embedding_model(inputs)
        return self.dense_layers(feature_embedding)
class NewsModel(keras.Model):

    def __init__(self, embedding_dimension : int = 32):
        super().__init__()


        # Define the news ID model.
        self.news_embeddings = keras.Sequential([
            # The news ID model is built the same way as the user ID model, just with different vocab and input dimensions.
            keras.layers.StringLookup(vocabulary = unique_news_ids),
            keras.layers.Embedding(input_dim = len(unique_news_ids) + 1, output_dim = embedding_dimension)
        ])

        max_title_tokens=1000

        self.title_text_embedding = keras.Sequential([
            keras.layers.TextVectorization(max_tokens=max_title_tokens),
            keras.layers.Embedding(max_title_tokens, embedding_dimension, mask_zero=True),
            keras.layers.GlobalAveragePooling1D()
        ])

        
        max_abstract_tokens=5000
        self.abstract_text_embedding = keras.Sequential([
            keras.layers.TextVectorization(max_tokens=max_abstract_tokens),
            keras.layers.Embedding(max_abstract_tokens, embedding_dimension, mask_zero=True),
            keras.layers.GlobalAveragePooling1D()
        ])

        self.category_embeddings = keras.Sequential([
            # Here we exclude the mask token as to better handle OOV items like new users or items.
            keras.layers.StringLookup(vocabulary = unique_categories),

            # Our final layer in the user ID model is an embedding layer which takes the IDs index
            # and creates a dense vector representation of it.
            keras.layers.Embedding(input_dim = len(unique_categories) + 1, output_dim = embedding_dimension)
        ])

        self.subcategory_embeddings = keras.Sequential([
            # Here we exclude the mask token as to better handle OOV items like new users or items.
            keras.layers.StringLookup(vocabulary = unique_subcategories),

            # Our final layer in the user ID model is an embedding layer which takes the IDs index
            # and creates a dense vector representation of it.
            keras.layers.Embedding(input_dim = len(unique_subcategories) + 1, output_dim = embedding_dimension)
        ])

    def call(self, inputs):
        return tf.concat([
            self.news_embeddings(inputs['news_id']),
            # self.title_text_embedding(inputs['title']),
            # self.abstract_text_embedding(inputs['abstract']),
            self.category_embeddings(inputs['category']),
            self.subcategory_embeddings(inputs['sub_category'])
        ], axis=1)

        # 160


class CandidateModel(keras.Model):
    
    def __init__(self, layer_sizes):

        super().__init__()

        self.embedding_model = NewsModel()
        self.dense_layers = keras.Sequential()

        # Use the ReLU activation for all but the last layer.
        for layer_size in layer_sizes[:-1]:
            self.dense_layers.add(keras.layers.Dense(layer_size, activation="relu"))

        # No activation for the last layer.
        for layer_size in layer_sizes[-1:]:
            self.dense_layers.add(keras.layers.Dense(layer_size))

    def call(self, inputs):
        # print(inputs)
        feature_embedding = self.embedding_model(inputs)
        return self.dense_layers(feature_embedding)
        

class MIND_model(tfrs.models.Model):
    """ 
    MIND model implementation for Tensorflow modelling with extra features!
    """

    def __init__(self, rating_weight :float, retrieval_weight : float, layer_sizes = None, embedding_dimension : float = 32): 

        super().__init__()

        self.query_model = QueryModel(layer_sizes)
        self.candidate_model = CandidateModel(layer_sizes)

        # Define the ratings model. As stated in documentation this can be as complicated as desired.
        self.rating_model = keras.Sequential([
            # Initialize dense layers of neurons with the rectified linear unit activation function.
            keras.layers.Dense(320, activation="relu"),
            keras.layers.Dense(256, activation="relu"),
            keras.layers.Dense(64, activation="relu"),
            keras.layers.Dense(1)
        ])

        # Now we define tasks
        # Set up the task of the recommender system.
        self.rating_task = tfrs.tasks.Ranking(
            # Select the mean squared error loss function and the rmse for the ranking task and the metrics.
            loss = keras.losses.MeanSquaredError(), # Can also swap this out for binary cross entropy, which might be nicer.
            metrics = [keras.metrics.RootMeanSquaredError()]
        )

        # self.retrieval_task = tfrs.tasks.Retrieval(
        #     metrics=tfrs.metrics.FactorizedTopK(
        #         candidates=news_ds.batch(128).map(self.candidate_model)
        #     )
        # )

        # Define loss weights
        self.rating_weight = rating_weight
        self.retrieval_weight = retrieval_weight

    def call(self, features):
        """
        Allows for the newsRankingModel to be called like a function. As an example, see the following:
        newsRankingModel((features["user_id"], features["movie_title"])). In the most simple iteration of this ranking model
        only the user and news ids are passed into the inputs argument. 

        Args:
            inputs (tuple) : Inputs is a tuple of features being placed in the call to be extracted with the models 
            that were defined in the __init__ method.

        Returns:
            rating (int) : Returns a rating created by passing the elements of `inputs` into the embedding models and
            the ratings model.
        """

        # Extract the user and news IDs.
        # get stuff from query model
        # get stuff from candidate model 
        # push through ratings        
        query_embeddings = self.query_model({
            "user_id" : features['user_id'], 
            "time" : features['time']
        })
        candidate_embeddings = self.candidate_model({
            "news_id": features['news_id'],
            "category" : features['category'], 
            "sub_category" : features['sub_category'],
            "title" : features["title"],
            'abstract' : features['abstract']
        })

        return (
            query_embeddings,
            candidate_embeddings,


            # Then apply multi-layered model to concatenation of user and news embeddings.
            self.rating_model(
                tf.concat([query_embeddings,candidate_embeddings ], axis=1)
            )

        )
    
    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:

        ratings = features['score']

        # Call the model to get the user and movie embeddings
        query_embeddings, candidate_embeddings, rating_predictions = self(features)

        # We compute the loss for each task.
        rating_loss = self.rating_task(
            labels=ratings,
            predictions=rating_predictions,
        )
        # retrieval_loss = self.retrieval_task(query_embeddings, candidate_embeddings)

        # And combine them using the loss weights.
        return (rating_loss)  # + retrieval_loss)

In [46]:
shuffled = tf_ds.shuffle(10_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(700_000)
test = shuffled.skip(700_000).take(300_000)
cached_train = train.shuffle(10_000).batch(1000).cache()
cached_test = test.batch(4096).cache()
model = MIND_model(layer_sizes=[32], rating_weight=1.0, retrieval_weight=1.0)
model.compile(optimizer=keras.optimizers.Adagrad(learning_rate=0.1))
one_layer_history = model.fit(cached_train, validation_data = cached_test, validation_freq=5,  epochs=3)
# accuracy = 
print(one_layer_history.history)
# print(f"Top-100 accuracy: {accuracy:.2f}.")


Epoch 1/3
{'news_id': <tf.Tensor 'data_2:0' shape=(None,) dtype=string>, 'category': <tf.Tensor 'data_3:0' shape=(None,) dtype=string>, 'sub_category': <tf.Tensor 'data_4:0' shape=(None,) dtype=string>, 'title': <tf.Tensor 'data_5:0' shape=(None,) dtype=string>, 'abstract': <tf.Tensor 'data_6:0' shape=(None,) dtype=string>}
{'news_id': <tf.Tensor 'data_2:0' shape=(None,) dtype=string>, 'category': <tf.Tensor 'data_3:0' shape=(None,) dtype=string>, 'sub_category': <tf.Tensor 'data_4:0' shape=(None,) dtype=string>, 'title': <tf.Tensor 'data_5:0' shape=(None,) dtype=string>, 'abstract': <tf.Tensor 'data_6:0' shape=(None,) dtype=string>}


/opt/conda/lib/python3.11/site-packages/keras/src/layers/layer.py:359: UserWarning: `build()` was called on layer 'mind_model_16', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


{'news_id': <tf.Tensor 'data_2:0' shape=(None,) dtype=string>, 'category': <tf.Tensor 'data_3:0' shape=(None,) dtype=string>, 'sub_category': <tf.Tensor 'data_4:0' shape=(None,) dtype=string>, 'title': <tf.Tensor 'data_5:0' shape=(None,) dtype=string>, 'abstract': <tf.Tensor 'data_6:0' shape=(None,) dtype=string>}
{'news_id': <tf.Tensor 'data_2:0' shape=(None,) dtype=string>, 'category': <tf.Tensor 'data_3:0' shape=(None,) dtype=string>, 'sub_category': <tf.Tensor 'data_4:0' shape=(None,) dtype=string>, 'title': <tf.Tensor 'data_5:0' shape=(None,) dtype=string>, 'abstract': <tf.Tensor 'data_6:0' shape=(None,) dtype=string>}
700/700 ━━━━━━━━━━━━━━━━━━━━ 17s 17ms/step - loss: 0.0711 - regularization_loss: 0.0000e+00 - total_loss: 0.0712
Epoch 2/3
700/700 ━━━━━━━━━━━━━━━━━━━━ 11s 15ms/step - loss: 0.0104 - regularization_loss: 0.0000e+00 - total_loss: 0.0104
Epoch 3/3
700/700 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - loss: 0.0080 - regularization_loss: 0.0000e+00 - total_loss: 0.0080
{'loss': 

In [47]:
model.evaluate(cached_test, return_dict=True)


{'news_id': <tf.Tensor 'IteratorGetNext:3' shape=(None,) dtype=string>, 'category': <tf.Tensor 'IteratorGetNext:1' shape=(None,) dtype=string>, 'sub_category': <tf.Tensor 'IteratorGetNext:5' shape=(None,) dtype=string>, 'title': <tf.Tensor 'IteratorGetNext:7' shape=(None,) dtype=string>, 'abstract': <tf.Tensor 'IteratorGetNext:0' shape=(None,) dtype=string>}
74/74 ━━━━━━━━━━━━━━━━━━━━ 8s 46ms/step - loss: 0.0096 - regularization_loss: 0.0000e+00 - total_loss: 0.0097


{'loss': 0.0}

## Interpretation of results
Irrespective of our ability to make a complete Tensorflow model we still are able to train a model that can predict item ratings. Overall our results are quite good with our training loss sitting at 0.0080 and our testing loss at 0.0097.